github地址:[https://github.com/cheesezh/python_design_patterns](https://github.com/cheesezh/python_design_patterns)

## 题目

用代码模拟以下场景，一个游戏角色有生命力，攻击力，防御力等数据，在打Boss前后的数据值会发生变化，如果玩家挑战Boss失败，我们允许玩家可以将游戏数据恢复到与Boss决斗之前的状态。

## 基础版本

In [2]:
class GameRole():
    
    def __init__(self):
        self.vitality = 0
        self.attack = 0
        self.defense = 0
        
    def state_display(self):
        print("当前角色状态:")
        print("体力:",self.vitality)
        print("攻击:",self.attack)
        print("防御:",self.defense)
        
    def get_init_state(self):
        """
        从服务器或者本地磁盘加载初始状态
        """
        self.vitality = 100
        self.attack = 100
        self.defense = 100
        
    def fight(self):
        """
        与Boss决斗，数据损耗为0
        """
        self.vitality = 0
        self.attack = 0
        self.defense = 0
        
        
def main():
    mario = GameRole()  
    mario.get_init_state()  # 大战Boss前，获取初始角色状态
    mario.state_display()
    
    backup = GameRole()  # 保存游戏进度
    backup.vitality = mario.vitality
    backup.attack = mario.attack
    backup.defense = mario.defense
    
    mario.fight()  # 大战Boss
    mario.state_display()
    
    mario.vitality = backup.vitality  # 读取进度，重新来过
    mario.attack = backup.attack
    mario.defense = backup.defense
    
    mario.state_display()
    
main()

当前角色状态:
体力: 100
攻击: 100
防御: 100
当前角色状态:
体力: 0
攻击: 0
防御: 0
当前角色状态:
体力: 100
攻击: 100
防御: 100


### 点评
主要问题在于客户端调用：
1. 游戏角色细节暴露给了客户端，客户端需要知道游戏角色的生命力，攻击力，防御力等细节；
2. 客户端还要对游戏角色进行备份；
3. 如果以后增加“魔法值”或者修改现有数据，都要更改客户端；
4. 恢复角色的过程也存在同样问题；

## 备忘录模式
备忘录模式，在不破坏封装性的前提下，捕获一个对象的内部状态，并在该对象之外保存这个状态。这样以后就可将该对象恢复到原先保存的状态。主要包括三种类：
- 发起人(Originator)：负责创建一个备忘录Memento，用以记录当前时刻它的内部状态，并可使用备忘录恢复内部状态。Originator可以根据需要决定Memento存储Originator的哪些内部状态；
- 备忘录(Memento)：负责存储Originator对象的内部状态，并可防止Originator以外的其他对象访问备忘录Memento。备忘录有两个接口，Caretaker只能看到备忘录的窄接口，它只能将备忘录传递给其他对象。Originator可以看到一个宽接口，允许它访问返回到先前状态的所有数据；
- 管理者(Caretaker)：负责保存好备忘录Memento，不能对备忘录的内容进行操作或检查。


在题目的场景中，游戏角色类相当于Originator，使用同样的“游戏角色”实例“backup”来做备忘录，这在当需要保存全部信息时，是可以考虑的，而用clone的方式来实现Memento的状态保存可能是更好的方法，但是这样就相当于对上层应用开放了Originator的全部接口，这对于保存备份有时候并不合适。

那么当我们不需要保存全部信息以备使用时，需要怎么办？当我们需要保存的并不是全部信息，而只是部分信息，那么就应该有独立的备忘录类Memento，它只拥有需要保存的信息的属性。

## 改进版本——备忘录模式

In [4]:
class GameRole():
    
    def __init__(self):
        self.vitality = 0
        self.attack = 0
        self.defense = 0
        
    def state_display(self):
        print("当前角色状态:")
        print("体力:",self.vitality)
        print("攻击:",self.attack)
        print("防御:",self.defense)
        
    def get_init_state(self):
        """
        从服务器或者本地磁盘加载初始状态
        """
        self.vitality = 100
        self.attack = 100
        self.defense = 100
        
    def fight(self):
        """
        与Boss决斗，数据损耗为0
        """
        self.vitality = 0
        self.attack = 0
        self.defense = 0
        
    def save_state(self):
        """
        新增“保存状态”的方法
        """
        return RoleStateMemento(self.vitality, self.attack, self.defense)
    
    def recovery_state(self, memento):
        """
        新增“恢复状态”的方法
        """
        self.vitality = memento.vitality
        self.attack = memento.attack
        self.defense = memento.defense
        

class RoleStateMemento():
    """
    游戏角色状态存储箱
    """
    def __init__(self, vitality, attack, defense):
        self.vitality = vitality
        self.attack = attack
        self.defense = defense
        

class RoleStateCaretaker():
    """
    角色状态管理者类
    """
    def __init__(self):
        self.memento = None
        

def main():
    mario = GameRole()
    mario.get_init_state()
    mario.state_display()
    
    state_admin = RoleStateCaretaker()
    state_admin.memento = mario.save_state()  # 游戏角色的保存对客户端是透明的
    
    mario.fight()
    mario.state_display()
    
    mario.recovery_state(state_admin.memento)  # 游戏角色的恢复对客户端是透明的
    mario.state_display()
    
main()

当前角色状态:
体力: 100
攻击: 100
防御: 100
当前角色状态:
体力: 0
攻击: 0
防御: 0
当前角色状态:
体力: 100
攻击: 100
防御: 100


### 点评
将需要保存的细节封装在Memento类中，那一天要更改保存的细节也不用影响客户端。

Memento模式比较适用于功能复杂的，但需要维护或记录属性历史的类，或者需要保存的属性只是众多属性中的一小部分时，Originator可以根据保存的Memento信息还原到前一状态。

命令模式也有类似的撤销作用，如果在某个系统中使用命令模式时，需要实现命令的撤销功能，那么命令模式可以使用备忘录模式来存储可撤销操作的状态。有时候一些对象的内部信息必须保存在对象以外的地方，但是必须要由对象自己读取，这时，使用备忘录可以把复杂的对象内部信息对其他的对象屏蔽起来，从而可以恰当的保持封装的边界。

当角色的状态改变的时候，有可能这个状态无效，这时候就可以使用暂时存储起来的备忘录将状态复原。